In [1]:
import requests
import os
import json

In [2]:
base_url = 'https://fantasy.premierleague.com/api/' 

### Update collection of PL players 

In [3]:
r = requests.get(base_url+'bootstrap-static/').json()

el_type_conversion = {1: 'goalkeeper', 2: 'defender', 3: 'midfielder', 4: 'forward'}

player_collection = {}
for el in r['elements']:
    
    player_collection[el['id']] = {
        'first_name': el['first_name'],
        'second_name': el['second_name'],
        'position': el_type_conversion[el['element_type']]
    }

with open('player_collection.json', 'w') as pc:
    json.dump(player_collection, pc)

### Detailed data on individual PL player

In [28]:
player_id = 131
player_url = f'element-summary/{player_id}'
res = requests.get(os.path.join(base_url, player_url)).json()

### Get team id, team name and manager name from all managers in Tangon

In [4]:
league_id = '5303'
mini_league_url = f'leagues-classic/{league_id}/standings/'

res = requests.get(os.path.join(base_url, mini_league_url)).json()
standings = res['standings']['results']

tango_managers = {}
for manager in standings:
    tango_managers[manager['entry']] = {'name': manager['player_name'], 'team': manager['entry_name']}

with open('tango_managers.json', 'w') as tm:
    json.dump(tango_managers, tm)

### Calculation coach of the month + team value delta

In [5]:
tango_managers = json.load(open('tango_managers.json'))
start_gameweek = 1
end_gameweek = 1

scores = {}

for manager_id in tango_managers:
    manager = tango_managers[manager_id]
    manager_history_url = f'entry/{manager_id}/history'
    
    res = requests.get(os.path.join(base_url, manager_history_url)).json()
    total_points = 0
    
    for gameweek in res['current']:
        if gameweek['event'] in range(start_gameweek, end_gameweek+1):
            total_points += gameweek['points']
            
            if gameweek['event'] == start_gameweek:
                start_value = gameweek['value'] / 10.0
            if gameweek['event'] == end_gameweek:
                end_value = gameweek['value'] / 10.0
    
    scores[manager['name']] = {'points': total_points, 'end_team_value': end_value, 'team_value_delta': end_value - start_value}
    

with open('coach_of_the_month_and_team_value.json', 'w') as c:
    json.dump(scores, c)

### Calculation captain points, points on bench and number of transfers

In [6]:
start_of_month_gw = 1
end_of_month_gw = 1
tango_managers = json.load(open('tango_managers.json'))

scores = {}

for manager_id in tango_managers:
    manager = tango_managers[manager_id]
    scores[manager['team']] = {
        'points_on_bench_month': 0,
        'points_on_bench_total': 0,
        'number_of_transfers_month': 0,
        'number_of_transfers_total': 0,
        'captain_points_month': 0,
        'captain_points_total': 0
    } 
    for gw in range(1, end_of_month_gw+1):
        manager_pick_gw_url = f'entry/{manager_id}/event/{gw}/picks'
        res = requests.get(os.path.join(base_url, manager_pick_gw_url)).json()
        
        # look up which player is captain
        picks = res['picks']
        for pick in picks:
            if pick['is_captain']:
                captain = pick['element']
        
        # get data from captain
        player_url = f'element-summary/{captain}'
        res_captain = requests.get(os.path.join(base_url, player_url)).json()
        captain_history = res_captain['history']
        
        for captain_fixture in captain_history:
            scores[manager['team']]['captain_points_total'] += captain_fixture['total_points']
            if captain_fixture['round'] >= start_of_month_gw:
                 scores[manager['team']]['captain_points_month'] += captain_fixture['total_points']
        
        entry = res['entry_history']
        
        # total calculation
        scores[manager['team']]['points_on_bench_total'] += entry['points_on_bench']
        scores[manager['team']]['number_of_transfers_total'] += entry['event_transfers']
        
        # month calculation
        if gw >= start_of_month_gw:
            scores[manager['team']]['points_on_bench_month'] += entry['points_on_bench']
            scores[manager['team']]['number_of_transfers_month'] += entry['event_transfers']

with open('transfers_captain_points_and_points_on_bench.json', 'w') as t:
    json.dump(scores, t)

### Best bus, midfield, attack, points on bench, goals and assists

In [72]:
start_of_month_gw = 1
end_of_month_gw = 1
bench_included = False
captain_points_included = False

tango_managers = json.load(open('tango_managers.json'))
player_collection = json.load(open('player_collection.json'))

scores = {}

for manager_id in tango_managers:
    manager = tango_managers[manager_id]
    
    scores[manager['team']] = {
        'goalkeeper': {
            'total': 0,
            'month': 0
        },
        'defender': {
            'total': 0,
            'month': 0
        },
        'midfielder': {
            'total': 0,
            'month': 0
        },
        'forward': {
            'total': 0,
            'month': 0
        },
        'bench': {
            'total': 0,
            'month': 0
        },
        'goals': {
            'total': 0,
            'month': 0
        },
        'assists': {
            'total': 0,
            'month': 0
        },
        'expected_goals': {
            'total': 0.0,
            'month': 0.0
        },
        'expected_assists': {
            'total': 0.0,
            'month': 0.0
        }
    }
    
    for gw in range(1, end_of_month_gw+1):
        manager_pick_gw_url = f'entry/{manager_id}/event/{gw}/picks'
        res = requests.get(os.path.join(base_url, manager_pick_gw_url)).json()
        
        picks = res['picks']
        for idx, pick in enumerate(picks):
            # seems that picks are sorted by position on field with bench last, hence idx <= 10
            position = player_collection[str(pick['element'])]['position']

            # scalar value (1 = normal, 2 = captain and assume 3 = triple captain)
            multiplier = pick['multiplier']
            
            if multiplier == 2:
                multiplier = 1 if not captain_points_included else multiplier
            
            player_url = f"element-summary/{pick['element']}"
            
            # list of games this season with detailed data. Assume sorted hence simple indexing,
            # however if not sorted (will find out after gw2) then we can use 'round' key probably
            player_history = requests.get(os.path.join(base_url, player_url)).json()['history']
            player_history_gw = player_history[gw-1]
            
            if multiplier == 0:
                scores[manager['team']]['bench']['total'] += player_history_gw['total_points']
                if gw >= start_of_month_gw:
                    scores[manager['team']]['bench']['month'] += player_history_gw['total_points']

                multiplier = 1 if bench_included else multiplier

            scores[manager['team']][position]['total'] += player_history_gw['total_points'] * multiplier

            scores[manager['team']]['goals']['total'] += player_history_gw['goals_scored']
            scores[manager['team']]['assists']['total'] += player_history_gw['assists']
            scores[manager['team']]['expected_goals']['total'] += float(player_history_gw['expected_goals'])
            scores[manager['team']]['expected_assists']['total'] += float(player_history_gw['expected_assists'])
            if gw >= start_of_month_gw:
                scores[manager['team']][position]['month'] += player_history_gw['total_points'] * multiplier
                
                scores[manager['team']]['goals']['month'] += player_history_gw['goals_scored']
                scores[manager['team']]['assists']['month'] += player_history_gw['assists']
                scores[manager['team']]['expected_goals']['month'] += float(player_history_gw['expected_goals'])
                scores[manager['team']]['expected_assists']['month'] += float(player_history_gw['expected_assists'])
    
    scores[manager['team']]['expected_goals']['total'] = round(scores[manager['team']]['expected_goals']['total'], 2) 
    scores[manager['team']]['expected_goals']['month'] = round(scores[manager['team']]['expected_goals']['month'], 2) 
    scores[manager['team']]['expected_assists']['total'] = round(scores[manager['team']]['expected_assists']['total'], 2) 
    scores[manager['team']]['expected_assists']['month'] = round(scores[manager['team']]['expected_assists']['month'], 2)

with open('bus_midfield_attack_bench_goals_assists_xg_xa.json', 'w') as b:
    json.dump(scores, b)

### Least and most amount of points

In [88]:
start_of_month_gw = 1
end_of_month_gw = 1
tango_managers = json.load(open('tango_managers.json'))

scores = {}

for manager_id in tango_managers:
    manager = tango_managers[manager_id]
    
    manager_history_url = f'entry/{manager_id}/history'
    manager_history = requests.get(os.path.join(base_url, manager_history_url)).json()
    
    min_points_total = None
    min_points_month = None
    max_points_total = None
    max_points_month = None
    
    manager_scores = {
        'min_points_total': None,
        'max_points_total': None,
        'min_points_month': None,
        'max_points_month': None
    }
    
    for gw in manager_history['current']:
        if gw['event'] == 1:
            manager_scores['min_points_total'] = gw['total_points']
            manager_scores['max_points_total'] = gw['total_points']
        if gw['event'] == start_of_month_gw:
            manager_scores['min_points_month'] = gw['total_points']   
            manager_scores['max_points_month'] = gw['total_points']   
        if gw['event'] > 1:
            if gw['total_points'] < manager_scores['min_points_total']:
                manager_scores['min_points_total'] = gw['total_points']
            if gw['total_points'] > manager_scores['max_points_total']:
                manager_scores['max_points_total'] = gw['total_points']
            
            if gw['event'] in range(start_of_month_gw, end_of_month_gw+1):
                if gw['total_points'] < manager_scores['min_points_month']:
                    manager_scores['min_points_month'] = gw['total_points']
                if gw['total_points'] > manager_scores['max_points_month']:
                    manager_scores['max_points_month'] = gw['total_points']
    
    scores[manager['team']] = manager_scores

with open('least_and_most_points_total_and_month.json', 'w') as l:
    json.dump(scores, l)            